In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dataloader
from tqdm import tqdm

from model_architecture import Network

In [2]:
PROPOSAL_SIZE = (64, 64)
batch_size = 200

dataset_train = dataloader.PotholeDataset('./Potholes/annotated-images/', './Potholes/proposals/', './Potholes/annotated-images/', proposals_per_batch=batch_size, proposal_size=PROPOSAL_SIZE, split='train')
dataset_val = dataloader.PotholeDataset('./Potholes/annotated-images/', './Potholes/proposals/', './Potholes/annotated-images/', proposals_per_batch=batch_size, proposal_size=PROPOSAL_SIZE, split='val')
dataset_test = dataloader.PotholeDataset('./Potholes/annotated-images/', './Potholes/proposals/', './Potholes/annotated-images/', proposals_per_batch=batch_size, proposal_size=PROPOSAL_SIZE, split='test')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)

model = Network()
model.apply(init_weights)
model.to(device)
#Initialize the optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.1)

cuda


In [4]:
def train(model, optimizer, num_epochs=10):
    def loss_fun(output, target):
        return F.cross_entropy(output, target)
    
    out_dict = {
              'train_acc': [],
              'val_acc': [],
              'train_loss': [],
              'val_loss': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        train_correct = 0
        train_loss = []
        # for minibatch_no, (data, target) in tqdm(enumerate(dataset), total=len(dataset)):
        for idx, (single_image_dict) in tqdm(enumerate(dataset_train), total=len(dataset_train)):
            # for proposal, label, proposal_image in zip(single_image_dict['proposals'], single_image_dict['labels'], single_image_dict['proposal_images']):
            proposal_image, label = single_image_dict['proposal_images'].to(device), single_image_dict['labels'].to(device)
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(proposal_image)
            #Compute the loss

            # print(f"Proposal shape: {single_image_dict['proposals'].shape if single_image_dict['proposals'] is not None else 'None'}")
            # print(f"Proposal image shape: {proposal_image.shape if proposal_image is not None else 'None'}")
            # print(f"Label: {label}")
            # print(f"Label shape: {label.shape if isinstance(label, torch.Tensor) else 'Not a Tensor'}")

            loss = loss_fun(output, label)
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (label==predicted).sum().cpu().item()

        #Comput the test accuracy
        val_loss = []
        val_correct = 0
        model.eval()
        for single_val_dict in dataset_val:
            # for proposal_val, label_val, proposal_image_val in zip(single_val_dict['proposals'], single_val_dict['labels'], single_val_dict['proposal_images']):
            proposal_image_val, label_val = single_val_dict['proposal_images'].to(device), single_val_dict['labels'].to(device)
            with torch.no_grad():
                output = model(proposal_image_val)

            val_loss.append(loss_fun(output, label_val).cpu().item())
            predicted = output.argmax(1)
            val_correct += (label_val==predicted).sum().cpu().item()

        out_dict['train_acc'].append(train_correct/len(dataset_train)/batch_size)
        out_dict['val_acc'].append(val_correct/len(dataset_val)/batch_size)
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['val_loss'].append(np.mean(val_loss))

        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(val_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['val_acc'][-1]*100:.1f}%") # Dividing by 5 because of the batch_size
        
    return out_dict

In [5]:
train(model, optimizer)

  0%|          | 0/206 [00:00<?, ?it/s]

c:\Users\samer\Documents\Intro to deep learning for computer vision\pothole_detector\dataloader.py:153: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels[selected_indices])
 10%|█         | 1/10 [01:37<14:35, 97.24s/epoch]

Loss train: 0.437	 test: 0.415	 Accuracy train: 82.2%	 test: 82.8%


 20%|██        | 2/10 [03:13<12:52, 96.55s/epoch]

Loss train: 0.419	 test: 0.414	 Accuracy train: 82.1%	 test: 82.6%


...

 90%|█████████ | 9/10 [15:14<01:44, 104.99s/epoch]

Loss train: 0.413	 test: 0.410	 Accuracy train: 82.1%	 test: 82.7%


100%|██████████| 10/10 [16:57<00:00, 101.78s/epoch]

Loss train: 0.413	 test: 0.409	 Accuracy train: 82.1%	 test: 82.8%


{'train_acc': [0.8215776699029127,
  0.8210679611650485,
  0.8200242718446602,
  0.8211407766990291,
  0.8216019417475728,
  0.8196116504854369,
  0.8202669902912622,
  0.8209708737864078,
  0.8209951456310679,
  0.8210679611650485],
 'val_acc': [0.8280681818181819,
  0.8260227272727273,
  0.8289772727272726,
  0.8273863636363636,
  0.8268181818181819,
  0.82625,
  0.8279545454545455,
  0.8270454545454545,
  0.8273863636363636,
  0.8276136363636364],
 'train_loss': [0.4371206039942584,
  0.4185764163153843,
  0.41646292487394465,
  0.41417044516906,
  0.41311233266464714,
  0.4147604938270976,
  0.4139543606818301,
  0.41314644793283595,
  0.41301094053439724,
  0.41297246602553767],
 'val_loss': [0.4150359366427768,
  0.41359441727399826,
  0.4089807021346959,
  0.4097648798064752,
  0.41002653539180756,
  0.41066586700352753,
  0.4089528342539614,
  0.4097812033512376,
  0.4095045890320431,
  0.40940282426097174]}